In [3]:
import torch
_ = torch.tensor([0.2126, 0.7152, 0.0722], names=['c'])

In [ ]:
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])
x = weights.unsqueeze(-1).unsqueeze(-1)  

x.shape

print(weights.names) # no named columns yet
# shape [channels, 1, 1]



(None,)


In [5]:
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]

In [6]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [ ]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1) # shape [channels, 1, 1]
img_weights = (img_t * unsqueezed_weights) # shape [channels, rows, columns], after broadcasting
batch_weights = (batch_t * unsqueezed_weights) # shape [batch, channels, rows, columns], after broadcasting
img_gray_weighted = img_weights.sum(-3) # shape [rows, columns]
batch_gray_weighted = batch_weights.sum(-3) # shape [batch, rows, columns], so this is for each pixel in each image in the batch, we sum over channels based on the weights, this is Luminance calculation
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [6]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [7]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [20]:
img_named =  img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [9]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [10]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [11]:
try:
    gray_named = (img_named[..., :3] * weights_named).sum('channels')
except Exception as e:
    print(e)

Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.


In [12]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

In [26]:
x = torch.tensor([[1, 2, 3]], names=['rows', 'columns'])
x.shape

torch.Size([1, 3])

In [27]:
y = x.squeeze()

In [30]:
y.shape
y [0] = 1000 # modify the first element, this will change the X as well since squeeze is esentially a view

In [31]:
x

tensor([[1000,    2,    3]], names=('rows', 'columns'))

In [32]:
y

tensor([1000,    2,    3], names=('columns',))